# CS285 Fall 2022 Homework 5

@ Author Juanwu Lu

@ Date  Nov-14-22

&copy; UC Berkeley

In [4]:
from __future__ import annotations

import pandas as pd

In [5]:
# Common Tensorboard Data Preprocessing function
def _process_df(df: pd.DataFrame, name: str = None) -> None:
    assert ('Step' in df.columns) and ('Value' in df.columns)
    df.sort_values(by=['Step'], inplace=True)
    df.rename(
        columns={'Step': 'Time Step', 'Value': name or 'Value'},
        inplace=True
    )
    if 'Wall time' in df.columns:
        df.drop(labels=['Wall time'], axis=1, inplace=True)

## Problem 1: "Unspervised" RND and exploration performance

- <input type="checkbox"> Implement the RND algorithm
- <input type="checkbox"> Use the `argmax` policy regarding exploration critic to generate state-action tuples
- <input type="checkbox"> __Pick two of the three envs and compare RND exploration to random one__
- <input type="checkbox"> Implement a separate exploration strategy of your own choice.

## Problem 2: Offline learning on exploration data

- <input type="checkbox"> Implement the conservative Q-learning algorithm
- <input type="checkbox"> Use the CQL critic as the extrinsic critic and compare it to a standard DQN critic.
- <input type="checkbox">

## Problem 3: "Supervised" exploration with mixed reward bonuses

- <input type="checkbox">

## Problem 4: Offline Learning with AWAC

## Problem 5: Offline Learning with IQL